In [1]:
from typing import (List, Tuple)

import pennylane as qml
from pennylane import numpy as np
from pennylane import PauliX, PauliY, PauliZ

import matplotlib.pyplot as plt

from vqls import *

In [ ]:
"""Unit tests for getting LS2."""
Acoeffs, Aterms = A_Ising_num(4, 1, 1, 0.1)
assert Acoeffs == [1.0, 1.0, 1.0, 1.0, 1.0, 0.1, 0.1, 0.1]
assert Aterms == ['IIII', 'XIII', 'IXII', 'IIXI', 'IIIX', 'ZZII', 'IZZI', 'IIZZ']

In [ ]:
"""Terms in effective Hamiltonian for second linear system."""
# Number of qubits
n = 3

# Constants
zeta = 1.0
eta = 1.0
J = 0.1

Acoeffs, Aterms = A_Ising_num(n, zeta, eta, J)


# Don't know why Rigetti code normalizes the coefficients for Ising inspired system
# BUT this does replicate the matrix shown in their code
Amat = A_to_num(n, Acoeffs, Aterms)
Anorm = np.linalg.norm(Amat, ord = 2)
Acoeffs = np.array(Acoeffs)
Acoeffs /= Anorm

print(np.real(A_to_num(n, Acoeffs, Aterms)))

In [2]:
# Here, we set the main hyper-parameters of the model

n_qubits = 4

zeta = 1.0
eta_ising = 1.0
J = 0.1

n_shots = 10**7
tot_qubits = n_qubits + 1
ancilla_idx = n_qubits
steps = 85
eta = 0.99
q_delta = 0.01
rng_seed = 0

In [10]:
# Coefficients of the linear combination A = c_0 A_0 + c_1 A_1 ...

c = np.array([1.0, 0.1, 0.1, 0.2])

A_terms = ["IIII", "XXIZ", "XIIX", "XZXI"]
A_mat = A_to_num(n_qubits, c, A_terms)
A_norm = np.linalg.norm(A_mat)
print(A_norm)

c = c / A_norm
norm_A_mat = A_to_num(n_qubits, c, A_terms)


print(np.linalg.norm(norm_A_mat))


'''
c, A_terms = A_Ising_num(n_qubits, zeta, eta_ising, J)

A_mat = A_to_num(n_qubits, c, A_terms)
A_norm = np.linalg.norm(A_mat, ord = 2)
c = np.array(c)
c /= A_norm

norm_A_mat = A_to_num(n_qubits, c, A_terms)


print(np.linalg.norm(norm_A_mat))
'''
        





def variational_block(weights):
    #Variational circuit mapping the ground state |0> to the ansatz state |x>.
    
    #for idx in range(n_qubits):
    #    qml.Hadamard(wires=idx)

    for idx, element in enumerate(weights):
        if idx > 3:
            break
        qml.RY(element, wires=idx)

    #
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 3:
            continue
        if idx > 7:
            break
        qml.RY(element, wires=idx%4)
        
    qml.CZ(wires=(1, 2))
    
    for idx, element in enumerate(weights):
        if idx <= 7:
            continue
        if idx == 8:
            continue
        if idx == 11:
            continue
        if idx > 11:
            break
        qml.RY(element, wires = idx%4)

    
    #
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 11:
            continue
        if idx > 15:
            break
        qml.RY(element, wires=idx%4)

    qml.CZ(wires=(1, 2))

    for idx, element in enumerate(weights):
        if idx <= 15:
            continue
        if ((idx == 16) or (idx == 19)):
            continue
        if idx > 19:
            break
        qml.RY(element, wires = idx%4)
    
        
    
    #
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 19:
            continue
        if idx > 23:
            break
        qml.RY(element, wires=idx%4)

    qml.CZ(wires=(1, 2))

    for idx, element in enumerate(weights):
        if idx <= 23:
            continue
        if ((idx == 24) or (idx == 27)):
            continue
        if idx > 27:
            break
        qml.RY(element, wires = idx%4)
    


    #
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 27:
            continue
        if idx > 31:
            break
        qml.RY(element, wires=idx%4)

    qml.CZ(wires=(1, 2))

    for idx, element in enumerate(weights):
        if idx <= 31:
            continue
        if ((idx == 32) or (idx == 35)):
            continue
        if idx > 35:
            break
        qml.RY(element, wires = idx%4)


    #
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 35:
            continue
        if idx > 39:
            break
        qml.RY(element, wires=idx%4)

    qml.CZ(wires=(1, 2))

    for idx, element in enumerate(weights):
        if idx <= 39:
            continue
        if ((idx == 40) or (idx == 43)):
            continue
        if idx > 43:
            break
        qml.RY(element, wires = idx%4)

        #
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 43:
            continue
        if idx > 47:
            break
        qml.RY(element, wires=idx%4)

    qml.CZ(wires=(1, 2))

    for idx, element in enumerate(weights):
        if idx <= 47:
            continue
        if ((idx == 48) or (idx == 51)):
            continue
        if idx > 51:
            break
        qml.RY(element, wires = idx%4)

    #
    
    qml.CZ(wires=(0, 1))
    qml.CZ(wires=(2, 3))

    for idx, element in enumerate(weights):
        if idx <= 51:
            continue
        if idx > 55:
            break
        qml.RY(element, wires=idx%4)

    qml.CZ(wires=(1, 2))

    for idx, element in enumerate(weights):
        if idx <= 55:
            continue
        if ((idx == 56) or (idx == 59)):
            continue
        if idx > 59:
            break
        qml.RY(element, wires = idx%4)
    








def U_b():
    """Unitary matrix rotating the ground state to the problem vector |b> = U_b |0>."""
    for idx in range(n_qubits):
        qml.Hadamard(wires=idx)



np.random.seed(rng_seed)
#w = q_delta * np.random.randn(7*n_qubits, requires_grad=True)
w = range(15*n_qubits)

drawer = qml.draw(variational_block)
print(drawer(w))

A_num = A_to_num(n_qubits, c, A_terms)
print(np.real(A_num))

condition_number = np.linalg.norm(A_num) * np.linalg.norm(np.linalg.inv(A_num))
print(condition_number)

4.1182523
0.99999994
0: ──RY(0.00)─╭●──RY(4.00)───────────────╭●──RY(12.00)───────────────╭●──RY(20.00)───────────────╭●
1: ──RY(1.00)─╰Z──RY(5.00)─╭●──RY(9.00)──╰Z──RY(13.00)─╭●──RY(17.00)─╰Z──RY(21.00)─╭●──RY(25.00)─╰Z
2: ──RY(2.00)─╭●──RY(6.00)─╰Z──RY(10.00)─╭●──RY(14.00)─╰Z──RY(18.00)─╭●──RY(22.00)─╰Z──RY(26.00)─╭●
3: ──RY(3.00)─╰Z──RY(7.00)───────────────╰Z──RY(15.00)───────────────╰Z──RY(23.00)───────────────╰Z

───RY(28.00)───────────────╭●──RY(36.00)───────────────╭●──RY(44.00)───────────────╭●──RY(52.00)───
───RY(29.00)─╭●──RY(33.00)─╰Z──RY(37.00)─╭●──RY(41.00)─╰Z──RY(45.00)─╭●──RY(49.00)─╰Z──RY(53.00)─╭●
───RY(30.00)─╰Z──RY(34.00)─╭●──RY(38.00)─╰Z──RY(42.00)─╭●──RY(46.00)─╰Z──RY(50.00)─╭●──RY(54.00)─╰Z
───RY(31.00)───────────────╰Z──RY(39.00)───────────────╰Z──RY(47.00)───────────────╰Z──RY(55.00)───

─────────────┤  
───RY(57.00)─┤  
───RY(58.00)─┤  
─────────────┤  
[[ 0.24282146  0.          0.          0.          0.          0.
   0.          0.          0.          0.02

In [11]:
dev_mu = qml.device("default.qubit", wires=tot_qubits)

@qml.qnode(dev_mu, interface="autograd")
def local_hadamard_test(weights, l=None, lp=None, j=None, part=None):

    # First Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # For estimating the imaginary part of the coefficient "mu", we must add a "-i"
    # phase gate.
    if part == "Im" or part == "im":
        qml.PhaseShift(-np.pi / 2, wires=ancilla_idx)

    # Variational circuit generating a guess for the solution vector |x>
    variational_block(weights)

    # Controlled application of the unitary component A_l of the problem matrix A.
    A_to_code(idx = l, ancilla_idx= ancilla_idx, terms = A_terms)

    # Adjoint of the unitary U_b associated to the problem vector |b>.
    # In this specific example Adjoint(U_b) = U_b.
    U_b()

    # Controlled Z operator at position j. If j = -1, apply the identity.
    if j != -1:
        qml.CZ(wires=[ancilla_idx, j])

    # Unitary U_b associated to the problem vector |b>.
    U_b()

    # Controlled application of Adjoint(A_lp).
    # In this specific example Adjoint(A_lp) = A_lp.
    A_to_code(idx= lp, ancilla_idx= ancilla_idx, terms= A_terms)

    # Second Hadamard gate applied to the ancillary qubit.
    qml.Hadamard(wires=ancilla_idx)

    # Expectation value of Z for the ancillary qubit.
    return qml.expval(qml.PauliZ(wires=ancilla_idx))

def mu(weights, l=None, lp=None, j=None):
    """Generates the coefficients to compute the "local" cost function C_L."""

    mu_real = local_hadamard_test(weights, l=l, lp=lp, j=j, part="Re")
    mu_imag = local_hadamard_test(weights, l=l, lp=lp, j=j, part="Im")

    return mu_real + 1.0j * mu_imag

def psi_norm(weights):
    """Returns the normalization constant <psi|psi>, where |psi> = A |x>."""
    norm = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            norm = norm + c[l] * np.conj(c[lp]) * mu(weights, l, lp, -1)

    return abs(norm)


def cost_loc(weights):
    """Local version of the cost function. Tends to zero when A|x> is proportional to |b>."""
    mu_sum = 0.0

    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            for j in range(0, n_qubits):
                mu_sum = mu_sum + c[l] * np.conj(c[lp]) * mu(weights, l, lp, j)

    mu_sum = abs(mu_sum)

    # Cost function C_L
    return 0.5 - 0.5 * mu_sum / (n_qubits * psi_norm(weights))

np.random.seed(rng_seed)
w = q_delta * np.random.randn(15*n_qubits, requires_grad=True)

#grad = qml.jacobian(cost_loc)(w)
var_grad_test = qml.grad(cost_loc, argnum=0)(w)

#print(grad)
print(var_grad_test)




opt = qml.GradientDescentOptimizer(eta)
#opt = qml.AdamOptimizer(stepsize=0.15, beta1 = 0.8)
#opt = qml.QNGOptimizer(eta, qml.metric_tensor(cost_loc, metric_tensor_fn = metric_fn))

grad_vals = []
cost_history = []
for it in range(steps):
    cost_grad = qml.grad(cost_loc, argnum=0)(w)[0]
    w, cost = opt.step_and_cost(cost_loc, w)

    
    
    print("Step {:3d}       Cost_L = {:9.7f}        Cost_Grad = {:9.9f}".format(it, cost, cost_grad))
    cost_history.append(cost)
    grad_vals.append(cost_grad)

print(np.var(grad_vals))

[-0.21919726 -0.14140583 -0.18137911 -0.1508166  -0.21921065 -0.14145059
 -0.18151536 -0.15088878  0.         -0.14142987 -0.18150931  0.
 -0.21919917 -0.14128957 -0.18138231 -0.15076325  0.         -0.14132151
 -0.18140015  0.         -0.21920678 -0.14143536 -0.18154519 -0.15093282
  0.         -0.14140414 -0.18153794  0.         -0.21919552 -0.14137085
 -0.18142716 -0.15071044  0.         -0.14141571 -0.18144093  0.
 -0.21920756 -0.14136988 -0.1815753  -0.15092498  0.         -0.1413715
 -0.18155513  0.         -0.21919444 -0.14133459 -0.18143231 -0.1507992
  0.         -0.14137808 -0.18138824  0.         -0.21916695 -0.14143596
 -0.18152361 -0.15081248  0.         -0.14143486 -0.18156644  0.        ]
Step   0       Cost_L = 0.4948204        Cost_Grad = -0.219197256
Step   1       Cost_L = 0.2987690        Cost_Grad = 0.019884817
Step   2       Cost_L = 0.1277390        Cost_Grad = -0.010326937
Step   3       Cost_L = 0.0761807        Cost_Grad = -0.000627195
Step   4       Cost_L = 

In [ ]:
print(cost_history[1])
print(cost_history[29])

In [ ]:
plt.style.use("seaborn")
plt.plot(cost_history, "g")
plt.ylabel("Cost function")
plt.xlabel("Optimization steps")
plt.show()

In [ ]:
plt.style.use("seaborn")
plt.plot(var_grad_vals, "g")
plt.ylabel("Variance of gradient")
plt.xlabel("Optimization steps")
plt.show()

In [ ]:
temp_1 = np.kron(qml.Hadamard.compute_matrix(), qml.Hadamard.compute_matrix())
temp_2 = np.kron(temp_1, qml.Hadamard.compute_matrix())
temp_3 = np.kron(temp_2, qml.Hadamard.compute_matrix())
#temp_4 = np.kron(temp_3, qml.Hadamard.compute_matrix())
#temp_5 = np.kron(temp_4, qml.Hadamard.compute_matrix())

zero_vector = np.ones(2)
zero_vector[1] = 0
zero = np.kron(zero_vector, np.kron(zero_vector, np.kron(zero_vector, zero_vector)))

b = temp_3.dot(zero)
print(temp_3.dot(zero))
print(np.linalg.norm(b))

In [ ]:
A_inv = np.linalg.inv(A_num)
x = np.dot(A_inv, b)

c_probs = np.real((x / np.linalg.norm(x)) ** 2)

In [ ]:
dev_x = qml.device("default.qubit", wires = n_qubits, shots = n_shots)

@qml.qnode(dev_x, interface="autograd")
def prepare_and_sample(weights):
    # Variational circuit generating a guess for the solution vector |x>
    variational_block(weights)

    # We assume that the system is measured in the computational basis.
    # then sampling the device will give us a value of 0 or 1 for each qubit (n_qubits)
    # this will be repeated for the total number of shots provided (n_shots)
    return qml.sample()

In [ ]:
raw_samples = prepare_and_sample(w)

# convert the raw samples (bit strings) into integers and count them
samples = []

for sam in raw_samples:
    samples.append(int("".join(str(bs) for bs in sam), base = 2))

q_probs = np.bincount(samples) / n_shots

In [ ]:
print("x_n^2 = \n", c_probs)

In [ ]:
print("|<x|n>|^2 = \n", q_probs)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 4))

ax1.bar(np.arange(0, 2 ** n_qubits), c_probs, color="blue")
ax1.set_xlim(-0.5, 2 ** n_qubits - 0.5)
ax1.set_xlabel("Vector space basis")
ax1.set_title("Classical probabilities")

ax2.bar(np.arange(0, 2 ** n_qubits), q_probs, color="green")
ax2.set_xlim(-0.5, 2 ** n_qubits - 0.5)
ax2.set_xlabel("Hilbert space basis")
ax2.set_title("Quantum probabilities")

plt.show()

In [ ]:
test_mat = np.array([[1, 1, 0 , 0], [1, 0, 0, 0], [0, 0, 1, 1], [0, 0, 1, 1]])

output = qml.pauli_decompose(test_mat, pauli = True)

print(output)
#print(list(output.values()))
#print(list(output.keys()))
print((output).to_mat(wire_order = [0, 1]))

n = 2
coefs = np.array([0.75, 1, 0.25, -0.25, 0.25])
terms = ["II", "IX", "IZ", "ZI", "ZZ"]

output_terms = list()

for term in terms:
    output_terms.append(qml.pauli.string_to_pauli_word(term))
    print(qml.pauli.string_to_pauli_word(term))

output_sentence = qml.pauli.PauliSentence(dict(zip(output_terms, coefs)))
print(output_sentence)
print((output_sentence).to_mat(wire_order = range(n)))

valid_mat = A_to_num(2, coefs, terms)
#print(test_mat)
print(np.real(valid_mat))

In [ ]:
print(qml.pauli.string_to_pauli_word("XY"))

In [ ]:
n_qubits = 2

c = np.array([0.2, 1])
A_terms = ["I", "ZX"]

print(A_to_code(1, A_terms))

#print(np.real(vqls.A_to_num(n_qubits, c, A_terms)))

#mat = np.zeros((dim, dim), dtype=np.complex64)
#print(np.real(mat))

